## Problem Statement 

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling 

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [1]:
import json 
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [24]:
# Opening JSON file 
f = open('/kaggle/input/automatic-ticket-classification/complaints-2021-05-14_08_16.json')# Write the path to your data file and load it 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [25]:
# Inspect the dataframe to understand the given data.
df.sample(10)


,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
12081,complaint-public-v2,complaint,3795863,0.0,None,75002,3795863,Managing an account,2020-08-13T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2020-08-14T12:00:00-05:00,None,Checking account,Yes,,Deposits and withdrawals,N/A
20239,complaint-public-v2,complaint,4064062,0.0,Older American,144XX,4064062,Managing an account,2021-01-12T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Phone,2021-02-12T12:00:00-05:00,None,Savings account,Yes,,Deposits and withdrawals,N/A
125,complaint-public-v2,complaint,3441607,0.0,None,92025,3441607,Problem caused by your funds being low,2019-11-15T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-11-18T12:00:00-05:00,None,Checking account,Yes,,Overdrafts and overdraft fees,N/A
33441,complaint-public-v2,complaint,3952127,0.0,None,08701,3952127,Closing your account,2020-11-09T12:00:00-05:00,NJ,...,Closed with non-monetary relief,JPMORGAN CHASE & CO.,Referral,2020-11-12T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Company closed your account,N/A
23218,complaint-public-v2,complaint,631103,0.0,None,236XX,631103,"Loan modification,collection,foreclosure",2013-12-12T12:00:00-05:00,VA,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2013-12-13T12:00:00-05:00,None,Conventional fixed mortgage,Yes,,None,N/A
5637,complaint-public-v2,complaint,4034803,0.0,None,19063,4034803,Closing your account,2020-12-28T12:00:00-05:00,PA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2020-12-28T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Company closed your account,Consent not provided
69612,complaint-public-v2,complaint,2907777,0.0,None,112XX,2907777,Improper use of your report,2018-05-16T12:00:00-05:00,NY,...,Closed with non-monetary relief,JPMORGAN CHASE & CO.,Web,2018-05-16T12:00:00-05:00,None,Credit reporting,Yes,I have 16 hard inquiries when I only had three...,Credit inquiries on your report that you don't...,Consent provided
19184,complaint-public-v2,complaint,842239,0.0,Older American,452XX,842239,"Loan modification,collection,foreclosure",2014-05-07T12:00:00-05:00,OH,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2014-05-12T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
4867,complaint-public-v2,complaint,3662049,0.0,None,85118,3662049,Closing your account,2020-05-20T12:00:00-05:00,AZ,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2020-05-21T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Company closed your account,N/A
21729,complaint-public-v2,complaint,3372949,0.0,None,100XX,3372949,Closing an account,2019-09-13T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-09-13T12:00:00-05:00,None,Checking account,Yes,"Hello, I had a Chase Secure Checking account a...",Company closed your account,Consent provided


In [33]:
df.shape

(78313, 22)

In [26]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [27]:
#Assign new column names
for column in df.columns:
    new_column = column.replace("_","").replace("source.", "")
    df.rename(columns={column: new_column}, inplace=True)

In [28]:
df.columns

Index(['index', 'type', 'id', 'score', 'tags', 'zipcode', 'complaintid',
       'issue', 'datereceived', 'state', 'consumerdisputed', 'product',
       'companyresponse', 'company', 'submittedvia', 'datesenttocompany',
       'companypublicresponse', 'subproduct', 'timely',
       'complaintwhathappened', 'subissue', 'consumerconsentprovided'],
      dtype='object')

In [29]:
df.sample(10)

,index,type,id,score,tags,zipcode,complaintid,issue,datereceived,state,...,companyresponse,company,submittedvia,datesenttocompany,companypublicresponse,subproduct,timely,complaintwhathappened,subissue,consumerconsentprovided
51274,complaint-public-v2,complaint,168089,0.0,None,89123,168089,Other,2012-10-09T12:00:00-05:00,NV,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2012-10-11T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
28576,complaint-public-v2,complaint,3477128,0.0,None,36016,3477128,Problem with a purchase shown on your statement,2019-12-25T12:00:00-05:00,AL,...,Closed with non-monetary relief,JPMORGAN CHASE & CO.,Web,2019-12-25T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,JP Morgan bank/ Chase Bank is reporting a cred...,Card was charged for something you did not pur...,Consent provided
13458,complaint-public-v2,complaint,698943,0.0,None,90045,698943,"Loan modification,collection,foreclosure",2014-02-04T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2014-02-04T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
7471,complaint-public-v2,complaint,3366877,0.0,None,30080,3366877,Incorrect information on your report,2019-09-07T12:00:00-05:00,GA,...,Closed with non-monetary relief,JPMORGAN CHASE & CO.,Web,2019-09-07T12:00:00-05:00,None,Credit reporting,Yes,,Information belongs to someone else,Consent not provided
32771,complaint-public-v2,complaint,3659725,0.0,None,94558,3659725,Managing an account,2020-05-19T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2020-05-19T12:00:00-05:00,None,Checking account,Yes,,Problem accessing account,Consent not provided
61506,complaint-public-v2,complaint,1478893,0.0,None,981XX,1478893,Other,2015-07-21T12:00:00-05:00,WA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2015-07-21T12:00:00-05:00,None,None,Yes,My case is regarding the auto rental collision...,None,Consent provided
49108,complaint-public-v2,complaint,4291309,0.0,Older American,70047,4291309,Managing an account,2021-04-12T12:00:00-05:00,LA,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2021-04-12T12:00:00-05:00,None,Savings account,Yes,,Funds not handled or disbursed as instructed,None
18472,complaint-public-v2,complaint,3628365,0.0,Older American,327XX,3628365,Problem with a purchase shown on your statement,2020-04-29T12:00:00-05:00,FL,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2020-04-29T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,On XX/XX/XXXX I purchase two airline tickets f...,Credit card company isn't resolving a dispute ...,Consent provided
37166,complaint-public-v2,complaint,63433,0.0,Older American,60202,63433,Settlement process and costs,2012-05-09T12:00:00-05:00,IL,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2012-05-10T12:00:00-05:00,None,Home equity loan or line of credit,Yes,,None,N/A
40871,complaint-public-v2,complaint,2805457,0.0,None,None,2805457,Managing an account,2018-02-06T12:00:00-05:00,OH,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-02-06T12:00:00-05:00,None,Checking account,Yes,"Hello, I began making purchases of digital cur...",Fee problem,Consent provided


In [32]:
(df.complaintwhathappened == "").sum()

57241

In [34]:
#Assign nan in place of blanks in the complaints column
df[df.complaintwhathappened == ""] = np.nan 

In [35]:
(df.complaintwhathappened == "").sum()

0

In [43]:
(df.complaintwhathappened == np.nan).sum()

0

In [46]:
#Remove all rows where complaints column is nan
df.dropna(subset=['complaintwhathappened'], inplace=True)

In [47]:
df.shape

(21072, 22)

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [50]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_text(text):
    # Make the text lowercase
    text = text.lower()
    
    # Remove text in square brackets using regular expression
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove punctuation using string library
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove words containing numbers using regular expression
    text = re.sub(r'\w*\d\w*', '', text)
    
    return text

In [49]:
#Write your function to Lemmatize the texts
def lemmatize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize each word using its part of speech (POS)
    lemmatized_words = []
    for word, pos in nltk.pos_tag(words):
        pos_letter = pos[0].lower() if pos[0].lower() in ['a', 'r', 'n', 'v'] else 'n'
        lemma = lemmatizer.lemmatize(word, pos=pos_letter)
        lemmatized_words.append(lemma)
    
    # Join the lemmatized words back into a sentence
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

In [53]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df['cleaned_complaints'] = df['complaintwhathappened'].apply(clean_text)

In [54]:
df['cleaned_complaints'].sample(10)

25442    i am outraged at the subpar level of service p...
3119     i have disputed this unknown account with both...
41819    i took line of credit from washington mutual o...
29558    i made arrangements with a chase representativ...
5534     xxxx xxxx xxxx good day to you all \ni have a ...
54383     wife and i have a mortgage held jointly howev...
25128    i am writing to jpmcb card services because i ...
29204    i have been banking with chase for almost ten ...
41771    i had a mortgage with washington mutual bank f...
5555     it appeared on my credit report that i have  x...
Name: cleaned_complaints, dtype: object

In [ ]:
df_clean

In [ ]:
#Write your function to extract the POS tags 

def pos_tag(text):
  # write your code here



df_clean["complaint_POS_removed"] =  #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].


In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text


In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the unigram frequency


In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the bigram frequency

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the trigram frequency

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer 



#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.


## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = #write the value you want to test out

#keep the random_state =40
nmf_model = #write your code here

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics


In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4



In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = #write your code to assign topics to each rows.

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean=df_clean.groupby('Topic').head(5)
df_clean.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:

#Write your code to get the Vector count


#Write your code here to transform the word vector to tf-idf

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics



